In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import tables
import flammkuchen as fl
from pathlib import Path
from tqdm import tqdm
import tifffile as tiff
import matplotlib.pyplot as plt

from fimpylab.core.lightsheet_experiment import LightsheetExperiment

import json
from scipy.optimize import quadratic_assignment
from lotr.pca import qap_sorting_and_phase

In [ ]:
master_path =  Path(r"\\funes\shared\experiments\E0040_motions_cardinal\v21\ls\h2b_wb")
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v29_29p1s\ls\1sec")
files = list(master_path.glob("*_f*"))
print(len(files))
print(files[2:5])

In [ ]:
fish_dir = files[2]
print(fish_dir)
suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')
suite2p_data.keys()


In [ ]:
for f in files[2:5]:
    try:
        fish_dir = f
        print(fish_dir)
        suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')

        traces = suite2p_data['traces']
        num_rois = np.shape(traces)[0]
        norm_traces = np.copy(traces)
        norm_traces = norm_traces.T
        sd = np.nanstd(norm_traces, 0)
        mean = np.nanmean(norm_traces, 0)
        norm_traces = norm_traces - mean
        norm_traces = norm_traces / sd
        norm_traces = norm_traces.T

        '''
        fig = plt.figure(figsize=(10,10))
        plt.imshow(norm_traces, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
        fig.tight_layout()
        fig_name = "norm_traces.jpg"
        fig.savefig(str(fish_dir / fig_name), dpi=300)
        '''

        corrmat = np.corrcoef(norm_traces)
        ng_corr_ind = np.unique(np.where(corrmat < -0.5)[0])
        #print(ng_corr_ind)
        corrmat_neg = corrmat[ng_corr_ind]

        norm_traces_neg = norm_traces[ng_corr_ind]
        '''
        fig1 = plt.figure(figsize=(10,10))
        plt.imshow(corrmat, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-1, vmax=1)
        fig1.tight_layout()
        fig_name = "norm_traces_corrmat1.jpg"
        fig1.savefig(str(fish_dir / fig_name), dpi=300)


        fig2 = plt.figure(figsize=(10,10))
        plt.imshow(corrmat_neg, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-1, vmax=1)
        fig2.tight_layout()
        fig_name = "norm_traces_corrmat_only_negative.jpg"
        fig2.savefig(str(fish_dir / fig_name), dpi=300)

        fig3 = plt.figure(figsize=(10,10))
        plt.imshow(norm_traces_neg, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
        fig3.tight_layout()
        fig_name = "norm_traces_only_neg_corr.jpg"
        fig3.savefig(str(fish_dir / fig_name), dpi=300)


        fig_anat = plt.figure(figsize=(5,5))
        plt.imshow(anatomy, cmap="gray_r")
        plt.axis('off')
        file_name = fish_dir / "suite2p_anatomy.jpg"
        '''

        anatomy = np.nanmean(suite2p_data['anatomy_stack'], 0)
        anatomy = np.rot90(anatomy, k=3)

        fig_all, ax = plt.subplots(2, 3, figsize=(15,10))
        ax[0,0].imshow(anatomy, cmap="gray_r")
        ax[0,0].axis('off')
        ax[0,0].set_title('Anatomy')

        ax[0,1].imshow(norm_traces, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
        ax[0,1].axis('off')
        ax[0,1].set_title('All traces')

        ax[0,2].imshow(corrmat, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-1, vmax=1)
        ax[0,2].axis('off')
        ax[0,2].set_title('Corrmat')

        
        rois_stack = suite2p_data['rois_stack']
        for i in range(np.shape(rois_stack)[0]):
            rois = np.rot90(rois_stack[i], k=3)
            rois = np.ma.masked_where(rois < 1, rois)
            ax[1,0].imshow(rois, cmap="rainbow")
        ax[1,0].axis('off')
        ax[1,0].set_title('ROIs')

        ax[1,1].imshow(norm_traces_neg, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
        ax[1,1].axis('off')
        ax[1,1].set_title('Negatively correlated traces')

        #ax[1,2].imshow(corrmat_neg, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-1, vmax=1)
        #ax[1,2].axis('off')
        #ax[1,2].set_title('Corrmat (negativelly correlated)')

        min_vals = np.min(corrmat, axis=1)
        roi_min_val = np.zeros_like(rois, dtype=float)
        for i in ng_corr_ind:
            #print(i)
            roi_min_val[np.where(rois == (i))] = 1

        roi_min_val = np.ma.masked_where(roi_min_val < 0.9, roi_min_val)
        ax[1,2].imshow(roi_min_val, cmap="gray_r", vmin=0, vmax=1)
        ax[1,2].axis('off')
        ax[1,2].set_title('ROIs (min corr value)')
        file_name = fish_dir / "fig_all05_un.jpg"
        fig_all.savefig(str(file_name), dpi=300)
    except:
        print("Error")

In [ ]:
for f in files:
    plt.close("all")
    fish_dir = f.parent
    #print(fish_dir)
    suite2p_data = fl.load(fish_dir / 'data_from_suite2p_unfiltered.h5')
    anatomy = suite2p_data['anatomy_stack'][0]
    anatomy = np.rot90(anatomy, k=3)
    max_val = np.percentile(anatomy, 99.9)
    fig_anat = plt.figure(figsize=(5,5))
    plt.imshow(anatomy, cmap="gray_r", vmax=max_val)
    file_name = fish_dir / "suite2p_anatomy.jpg"
    fig_anat.savefig(file_name, dpi=300)
    

In [ ]:
traces = suite2p_data['traces']
norm_traces = np.copy(traces)
norm_traces = norm_traces.T
sd = np.nanstd(norm_traces, 0)
mean = np.nanmean(norm_traces, 0)
norm_traces = norm_traces - mean
norm_traces = norm_traces / sd
norm_traces = norm_traces.T

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(norm_traces, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
fig.tight_layout()
fig_name = "norm_traces.jpg"
fig.savefig(str(fish_dir / fig_name), dpi=300)

In [ ]:
fish_files = list(master_path.glob("*_f*"))
for f in fish_files:
    exp = TwoPExperiment(f)
    
    or_path = f / 'original'
    num_planes = len(list(or_path.glob("*00*")))
    print(num_planes)
    for i in range(num_planes):
        fig_all, ax = plt.subplots(2, 1, figsize=(15,10))
        beh_log = exp.load_session_log(log_name='behavior_log', session_idx=i)
        tail_trace = beh_log['tail_sum']

    

In [ ]:
for f in files:
    
    #f= f.parent
    print(f)
    exp = LightsheetExperiment(f)
    
    fig_all, ax = plt.subplots(2, 1, figsize=(15,10), sharex=True, gridspec_kw={'height_ratios': [1,3]})
    beh_log = exp.behavior_log
    tail_trace = beh_log['tail_sum']
    beh_t = np.asarray(beh_log['t']) * 5

    suite2p_data = fl.load(f / 'data_from_suite2p_cells.h5')

    traces = suite2p_data['traces']
    num_rois = np.shape(traces)[0]
    norm_traces = np.copy(traces)
    norm_traces = norm_traces.T
    sd = np.nanstd(norm_traces, 0)
    mean = np.nanmean(norm_traces, 0)
    norm_traces = norm_traces - mean
    norm_traces = norm_traces / sd
    norm_traces = norm_traces.T
    print(np.shape(norm_traces))

    corrmat = np.corrcoef(norm_traces)
    print(np.shape(corrmat))
    norm_traces_neg = norm_traces[np.unique(np.where(corrmat < -0.7)[0])]
    print(np.shape(norm_traces_neg))
    anatomy = suite2p_data['anatomy_stack'][0]
    anatomy = np.rot90(anatomy, k=3)

    perm, com_phase = qap_sorting_and_phase(norm_traces_neg.T)
    sorted_traces = norm_traces_neg[perm]
    sorted_corrmat = np.corrcoef(sorted_traces)

    ax[0].plot(beh_t, tail_trace, color='k')
    ax[0].set_aspect('auto')
    ax[0].axis('off')
    ax[0].set_title('Tail sum')

    ax[1].imshow(sorted_traces, cmap="coolwarm", vmin=-2, vmax=2, aspect='auto')
    ax[1].axis('off')
    ax[1].set_title('Negatively correlated traces')

    file_name = f / "sorted_traces_220510_05.jpg"
    fig_all.savefig(str(file_name), dpi=300)


In [ ]:
fish_files = list(master_path.glob("*_f*"))[1:]


for f in fish_files:
    print(f)
    try:
        exp = TwoPExperiment(f)

        or_path = f / 'original'
        num_planes = len(list(or_path.glob("*00*")))
        print(num_planes)
        for i in range(num_planes):
            print(i)
            try:
                fig_all, ax = plt.subplots(2, 1, figsize=(15,10), sharex=True, gridspec_kw={'height_ratios': [1,3]})
                beh_log = exp.load_session_log(log_name='behavior_log', session_idx=i)
                tail_trace = beh_log['tail_sum']
                beh_t = np.asarray(beh_log['t']) * 3

                plane = '000' + str(i)
                plane_dir = f / 'suite2p'  / plane
                try:
                    suite2p_data = fl.load(plane_dir / 'data_from_suite2p_unfiltered.h5')
                except:
                    plane = '00' + str(i)
                    plane_dir = f / 'suite2p'  / plane
                    suite2p_data = fl.load(plane_dir / 'data_from_suite2p_unfiltered.h5')

                #traces = fl.load(plane_dir / "filtered_traces.h5", "/undetr")
                traces = suite2p_data['traces']
                num_rois = np.shape(traces)[0]
                norm_traces = np.copy(traces)
                norm_traces = norm_traces.T
                sd = np.nanstd(norm_traces, 0)
                mean = np.nanmean(norm_traces, 0)
                norm_traces = norm_traces - mean
                norm_traces = norm_traces / sd
                norm_traces = norm_traces.T

                corrmat = np.corrcoef(norm_traces)
                norm_traces_neg = norm_traces[np.unique(np.where(corrmat < -0.4)[0])]
                anatomy = suite2p_data['anatomy_stack'][0]
                anatomy = np.rot90(anatomy, k=3)

                perm, com_phase = qap_sorting_and_phase(norm_traces_neg.T)
                sorted_traces = norm_traces_neg[perm]
                sorted_corrmat = np.corrcoef(sorted_traces)

                ax[0].plot(beh_t, tail_trace, color='k')
                ax[0].set_aspect('auto')
                ax[0].axis('off')
                ax[0].set_title('Tail sum')

                ax[1].imshow(sorted_traces, cmap="coolwarm", vmin=-2, vmax=2, aspect='auto')
                ax[1].axis('off')
                ax[1].set_title('Negatively correlated traces')

                file_name = plane_dir / "fig_all3_220112_sorted_filtered_04.jpg"
                fig_all.savefig(str(file_name), dpi=300)
            except:
                print("Error")
    except:
        print("lol")

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# clustering negatively correlated tracess: 

fish_files = list(master_path.glob("*_f*"))

for f in files:
    fish_dir = f.parent
    print(fish_dir)
    suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')
    
    traces = suite2p_data['traces']
    num_rois = np.shape(traces)[0]
    norm_traces = np.copy(traces)
    norm_traces = norm_traces.T
    sd = np.nanstd(norm_traces, 0)
    mean = np.nanmean(norm_traces, 0)
    norm_traces = norm_traces - mean
    norm_traces = norm_traces / sd
    norm_traces = norm_traces.T
    
    corrmat = np.corrcoef(norm_traces)
    ng_corr_ind = np.unique(np.where(corrmat < -0.3)[0])
    #print((ng_corr_ind))
    corrmat_neg = corrmat[ng_corr_ind]
    
    norm_traces_neg = norm_traces[ng_corr_ind]
    k = 12
    if len(ng_corr_ind)>k:
        
        kmeans = KMeans(k)
        clusters_norm = kmeans.fit_transform(norm_traces_neg)
        clustered_traces_norm = np.zeros_like(norm_traces_neg)
        labels_k_norm = kmeans.predict(norm_traces_neg)
        labelsinds_norm = labels_k_norm.argsort()
        clustered_traces_norm = norm_traces_neg[labelsinds_norm[::-1]]

        anatomy = suite2p_data['anatomy_stack'][0]
        anatomy = np.rot90(anatomy, k=3)
        max_val = np.percentile(anatomy, 99.9)

        fig_all, ax = plt.subplots(1, 3, figsize=(15,10))
        ax[0].imshow(anatomy, cmap="gray_r", vmax=max_val)
        ax[0].axis('off')
        ax[0].set_title('Anatomy')

        ax[1].imshow(clustered_traces_norm, extent=[0,  500, 0, 500], vmin=-2, vmax=2, cmap="coolwarm")
        ax[1].set_xlabel("Time (sec)")
        ax[1].set_title("Clustered traces")
        ax[1].axis('off')

        rois = suite2p_data['rois_stack'][0]
        rois = np.rot90(rois, k=3)

        roi_map_clustered = np.zeros_like(rois)
        count=0
        print(num_rois, np.shape(labels_k_norm))
        for i in ng_corr_ind:
            roi_map_clustered[np.where(rois == (i))] = labels_k_norm[count] + 1
            count +=1
        roi_map_clustered = np.ma.masked_where(roi_map_clustered < 1, roi_map_clustered)
        ax[2].imshow(roi_map_clustered, cmap="tab20")
        ax[2].axis('off')
        ax[2].set_title('clustered ROIs')
        file_name = "fig_all_kmeans_k" + str(k) + ".jpg"
        fig_all.savefig(str(fish_dir / file_name), dpi=300)
    else:
        print('no traces')

In [ ]:
b = a[np.where(a < 0)[0]]

In [ ]:
b

In [ ]:
b

In [ ]:
a